In [74]:
# Se importan las librerías a usar.
import pandas as pd
import gzip
import json
from pandas import json_normalize

In [3]:
# Ruta de los archivos a utilizar
ruta_items = "/content/items_corregido.json.gz"
ruta_reviews = "/content/reviews_corregido.json.gz"
ruta_games = "/content/steam_games.json.gz"

In [15]:
# se crean los dataframes con los que se trabajará.
items = pd.read_json(ruta_items,compression="gzip")

In [62]:
reviews = pd.read_json(ruta_reviews,compression = "gzip")

In [30]:
games = pd.DataFrame(pd.read_json(ruta_games,compression="gzip",lines=True))

In [55]:
# Se crea una función para guardar los nuevos archivos corregidos

def guardar_archivo(DataFrame,RutaNuevoArchivo):
  ''' Convierte un Dataframe dado en json y lo comprime en formato gzip'''
  archivo_json = DataFrame.to_json()
  with gzip.open(RutaNuevoArchivo, "w") as f:
    f.write(archivo_json.encode('utf-8'))

# Limpieza games

In [27]:
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [31]:
# Se eliminan las filas cuyos datos sean todos nulos (ya que no nos proporcionan información sobre el juego).
games.dropna(how = "all", inplace = True)
# Se procede a eliminar las columnas con mayor número de datos incompletos o que se repiten, así como aquellos cuya información no resulta útil para la formulación de las consultas.
games.drop(["title","url","reviews_url","price","early_access","publisher","genres"], axis = 1, inplace = True)


In [34]:
# Se revisa la base de datos obtenida a partir de la limpieza y se cambian los nombres de las columnas.
games.rename(columns={"app_name":"name","tags":"genre","id":"id_game"},inplace=True)
games

,name,release_date,genre,specs,id_game,developer
88310,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],761140.0,Kotoshiro
88311,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",643980.0,Secret Level SRL
88312,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",670290.0,Poolians.com
88313,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],767400.0,彼岸领域
88314,Log Challenge,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",773570.0,None
...,...,...,...,...,...,...
120440,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",773640.0,"Nikita ""Ghost_RUS"""
120441,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",733530.0,Sacada
120442,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",610660.0,Laush Dmitriy Sergeevich
120443,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",658870.0,"xropi,stev3ns"


In [37]:
# Se busca la información obtenida hasta el momento.
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         32135 non-null  int64  
 1   name          32133 non-null  object 
 2   release_date  30068 non-null  object 
 3   genre         31972 non-null  object 
 4   specs         31465 non-null  object 
 5   id_game       32133 non-null  float64
 6   developer     28836 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.7+ MB


In [36]:
#Se reinician los índex
games.reset_index(inplace=True)

In [42]:
# Se crea un nuevo dataframe para los géneros y para las especificaciones.

games_genres = games[["id_game","genre"]]
games_specs = games[["id_game","specs"]]

# Se eliminan estas columnas del primer data set (ya que ya no son necesarias) así como la columna index.
games.drop(columns=["genre","specs","index"],inplace = True)

In [52]:
# Se desagregan los datos del género para cada uno de los juegos.
games_genres = games_genres.explode(column=["genre"],ignore_index=True)
games_specs = games_specs.explode(column="specs",ignore_index=True)

In [56]:
# Se guardan los archivos corregidos en nuevos documentos.
guardar_archivo(games,"/content/games_clean.json.gz")
guardar_archivo(games_genres,"/content/genres_games.json.gz")
guardar_archivo(games_specs,"/content/specs_games.json.gz")

# Limpieza Reviews

In [63]:
# Se visualizan los datos del data frame.
reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25761,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25762,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25763,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25764,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [64]:
# Se elimina la columna "user_url", ya que no será usada.
reviews.drop(columns="user_url",inplace=True)

In [77]:
# Con el propósito de separar cada una de las reviews, se normalizar la columna JSON en un DataFrame nuevo.
normalized_reviews = json_normalize(reviews["reviews"])

In [78]:
# Se combina el DataFrame normalizado con el DataFrame original
reviews = pd.concat([reviews, normalized_reviews], axis=1).drop('reviews', axis=1)
reviews

,user_id,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
25761,76561198306599751,"{'funny': '', 'posted': 'Posted May 31.', 'las...",None,None,None,None,None,None,None,None,None
25762,Ghoustik,"{'funny': '', 'posted': 'Posted June 17.', 'la...",None,None,None,None,None,None,None,None,None
25763,76561198310819422,"{'funny': '1 person found this review funny', ...",None,None,None,None,None,None,None,None,None
25764,76561198312638244,"{'funny': '', 'posted': 'Posted July 21.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 10.', 'la...","{'funny': '', 'posted': 'Posted July 8.', 'las...",None,None,None,None,None,None
